<a href="https://colab.research.google.com/github/Davs07/Android-Project/blob/main/Simpsons_CNN_LATESTVERSION2_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ACTIVIDAD GRUPAL 3: REDES NEURONALES CONVOLUCIONALES

---

En esta actividad, vamos a trabajar con Convolutional Neural Networks para resolver un problema de clasificación de imágenes. En particular, vamos a clasificar imágenes de personajes de la conocida serie de los Simpsons.

Como las CNN profundas son un tipo de modelo bastante avanzado y computacionalmente costoso, se recomienda hacer la práctica en Google Colaboratory con soporte para GPUs. En [este enlace](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d) se explica cómo activar un entorno con GPUs. *Nota: para leer las imágenes y estandarizarlas al mismo tamaño se usa la librería opencv. Esta ĺibrería está ya instalada en el entorno de Colab, pero si trabajas de manera local tienes que instalarla.*

<center><img src="https://i.imgur.com/i8zIGqX.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consiste en imágenes de personajes de los Simpsons extraídas directamente de capítulos de la serie. Este dataset ha sido recopilado por [Alexandre Attia](http://www.alexattia.fr/) y es más complejo que el dataset de  MNIST que hemos utilizado hasta ahora. Aparte de tener más clases (vamos a utilizar los 18 personajes con más imágenes), los personajes pueden aparecer en distintas poses, en distintas posiciones de la imagen o con otros personajes en pantalla (si bien el personaje a clasificar siempre aparece en la posición predominante).

El dataset de training puede ser descargado desde aquí:

[Training data](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219337&authkey=AMzI92bJPx8Sd60) (~500MB)

Por otro lado, el dataset de test puede ser descargado de aquí:

[Test data](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219341&authkey=ANnjK3Uq1FhuAe8) (~10MB)

Antes de empezar la práctica, se recomienda descargar las imágenes y echarlas un vistazo.


In [1]:
! pip install gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00


## Carga de los datos

In [2]:
import gradio as gr
import cv2
import os
import numpy as np
import keras
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import tensorflow as tf



In [ ]:
# Primero, bajamos los datos de entrenamiento
keras.utils.get_file(fname="simpsons_train.tar.gz",
                     origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219337&authkey=AMzI92bJPx8Sd60")

# Descomprimimos el archivo
!tar -xzf /root/.keras/datasets/simpsons_train.tar.gz -C /root/.keras/datasets


# Hacemos lo mismo con los datos de test
keras.utils.get_file(fname="simpsons_test.tar.gz",
                     origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219341&authkey=ANnjK3Uq1FhuAe8")
!tar -xzf /root/.keras/datasets/simpsons_test.tar.gz -C /root/.keras/datasets

375144448/523789527 ━━━━━━━━━━━━━━━━━━━━ 1:33 1us/step

In [ ]:
# Descomprimimos el archivo en tmp para visualizar
# !tar -xzf /root/.keras/datasets/simpsons_train.tar.gz -C /tmp/simpsons

In [ ]:
# Esta variable contiene un mapeo de número de clase a personaje.
# Utilizamos sólo los 18 personajes del dataset que tienen más imágenes.
MAP_CHARACTERS = {
    0: 'abraham_grampa_simpson', 1: 'apu_nahasapeemapetilon', 2: 'bart_simpson',
    3: 'charles_montgomery_burns', 4: 'chief_wiggum', 5: 'comic_book_guy', 6: 'edna_krabappel',
    7: 'homer_simpson', 8: 'kent_brockman', 9: 'krusty_the_clown', 10: 'lisa_simpson',
    11: 'marge_simpson', 12: 'milhouse_van_houten', 13: 'moe_szyslak',
    14: 'ned_flanders', 15: 'nelson_muntz', 16: 'principal_skinner', 17: 'sideshow_bob'
}

# Vamos a standarizar todas las imágenes a tamaño 64x64
IMG_SIZE = 100

In [ ]:
def load_train_set(dirname, map_characters, verbose=True):
    """Esta función carga los datos de training en imágenes.

    Como las imágenes tienen tamaños distintas, utilizamos la librería opencv
    para hacer un resize y adaptarlas todas a tamaño IMG_SIZE x IMG_SIZE.

    Args:
        dirname: directorio completo del que leer los datos
        map_characters: variable de mapeo entre labels y personajes
        verbose: si es True, muestra información de las imágenes cargadas

    Returns:
        X, y: X es un array con todas las imágenes cargadas con tamaño
                IMG_SIZE x IMG_SIZE
              y es un array con las labels de correspondientes a cada imagen
    """
    X_train = []
    y_train = []
    for label, character in map_characters.items():
        files = os.listdir(os.path.join(dirname, character))
        images = [file for file in files if file.endswith("jpg")]
        if verbose:
          print("Leyendo {} imágenes encontradas de {}".format(len(images), character))
        for image_name in images:
            image = cv2.imread(os.path.join(dirname, character, image_name))
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = image.reshape(IMG_SIZE, IMG_SIZE, 1)
            X_train.append(cv2.resize(image,(IMG_SIZE, IMG_SIZE)))
            y_train.append(label)
    return np.array(X_train), np.array(y_train)

In [ ]:
def load_test_set(dirname, map_characters, verbose=True):
    """Esta función funciona de manera equivalente a la función load_train_set
    pero cargando los datos de test."""
    X_test = []
    y_test = []
    reverse_dict = {v: k for k, v in map_characters.items()}
    for filename in glob.glob(dirname + '/*.*'):
        char_name = "_".join(filename.split('/')[-1].split('_')[:-1])
        if char_name in reverse_dict:
            image = cv2.imread(filename)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = image.reshape(IMG_SIZE, IMG_SIZE, 1)
            X_test.append(image)
            y_test.append(reverse_dict[char_name])
    if verbose:
        print("Leídas {} imágenes de test".format(len(X_test)))
    return np.array(X_test), np.array(y_test)

In [ ]:
# Cargamos los datos. Si no estás trabajando en colab, cambia los paths por
# los de los ficheros donde hayas descargado los datos.
DATASET_TRAIN_PATH_COLAB = "/root/.keras/datasets/simpsons"
DATASET_TEST_PATH_COLAB = "/root/.keras/datasets/simpsons_testset"

X, y = load_train_set(DATASET_TRAIN_PATH_COLAB, MAP_CHARACTERS)
X_t, y_t = load_test_set(DATASET_TEST_PATH_COLAB, MAP_CHARACTERS)

In [ ]:
# Vamos a barajar aleatoriamente los datos. Esto es importante ya que si no
# lo hacemos y, por ejemplo, cogemos el 20% de los datos finales como validation
# set, estaremos utilizando solo un pequeño número de personajes, ya que
# las imágenes se leen secuencialmente personaje a personaje.
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_t[11].reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
plt.show()

## Ejercicio

Utilizando Convolutional Neural Networks con Keras, entrenar un clasificador que sea capaz de reconocer personajes en imágenes de los Simpsons con una accuracy en el dataset de test de, al menos, **90%**. Redactar un informe analizando varias de las alternativas probadas y los resultados obtenidos.

A continuación se detallan una serie de aspectos orientativos que podrían ser analizados en su informe (no es necesario tratar todos ellos ni mucho menos, esto son ideas orientativas de aspectos que pueden explorar):

*   Análisis de los datos a utilizar.
*   Análisis de resultados, obtención de métricas de *precision* y *recall* por clase y análisis de qué clases obtienen mejores o peores resultados.
*   Análisis visual de los errores de la red. ¿Qué tipo de imágenes o qué personajes dan más problemas a nuestro modelo?
*   Comparación de modelos CNNs con un modelo de Fully Connected para este problema.
*   Utilización de distintas arquitecturas CNNs, comentando aspectos como su profundidad, hiperparámetros utilizados, optimizador, uso de técnicas de regularización, *batch normalization*, etc.
*   [ *algo más difícil* ] Utilización de *data augmentation*. Esto puede conseguirse con la clase [ImageDataGenerator](https://keras.io/preprocessing/image/#imagedatagenerator-class) de Keras.

Notas:
* Recuerda partir los datos en training/validation para tener una buena estimación de los valores que nuestro modelo tendrá en los datos de test, así como comprobar que no estamos cayendo en overfitting. Una posible partición puede ser 80 / 20.
* No es necesario mostrar en el notebook las trazas de entrenamiento de todos los modelos entrenados, si bien una buena idea seria guardar gráficas de esos entrenamientos para el análisis. Sin embargo, **se debe mostrar el entrenamiento completo del mejor modelo obtenido y la evaluación de los datos de test con este modelo**.
* Las imágenes **no están normalizadas**. Hay que normalizarlas como hemos hecho en trabajos anteriores.
* El test set del problema tiene imágenes un poco más "fáciles", por lo que es posible encontrarse con métricas en el test set bastante mejores que en el training set.

In [ ]:
# Normalización de los datos (valores entre 0 y 1)
X = X / 255.0
X_t = X_t / 255.0

# Cambiamos el formato de las etiquetas a one-hot encoding
from tensorflow.keras.utils import to_categorical

y_train_one_hot = to_categorical(y, num_classes=len(MAP_CHARACTERS))
y_test_one_hot = to_categorical(y_t, num_classes=len(MAP_CHARACTERS))

In [ ]:
# Importar las bibliotecas necesarias
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
import datetime
import os

In [ ]:
# Montar Google Drive para guardar los logs
from google.colab import drive
drive.mount('/content/drive')

# Definir la ruta en Google Drive para guardar los logs de TensorBoard
log_dir_base = "/content/drive/My Drive/Colab Notebooks/tensorboard_logs/"

In [ ]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/tensorboard_logs/

In [ ]:
# Crear el directorio si no existe
if not os.path.exists(log_dir_base):
    os.makedirs(log_dir_base)

# Modelo 1

In [ ]:
# ----------ARQUITECTURA PRIMER MODELO-------------

# Definir el primer modelo CNN
cnn_model = Sequential()

# Primera capa convolucional
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda capa convolucional
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

# Tercera capa convolucional
cnn_model.add(Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

# Aplanar la salida
cnn_model.add(Flatten())

# Capa completamente conectada
cnn_model.add(Dense(128, activation='relu'))

# Capa de salida
cnn_model.add(Dense(18, activation='softmax'))  # 18 clases de salida

# Compilar el modelo
cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Modelo 2

In [ ]:
# ----------ARQUITECTURA SEGUNDO MODELO-------------

# Definir el segundo modelo CNN
cnn_model2 = Sequential()

# Primera capa convolucional
cnn_model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
cnn_model2.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda capa convolucional
cnn_model2.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model2.add(MaxPooling2D(pool_size=(2, 2)))

# Tercera capa convolucional
cnn_model2.add(Conv2D(128, (3, 3), activation='relu'))
cnn_model2.add(MaxPooling2D(pool_size=(2, 2)))

# Añadir DropOut
cnn_model2.add(Dropout(0.5))

# Aplanar la salida
cnn_model2.add(Flatten())

# Capa completamente conectada
cnn_model2.add(Dense(256, activation='relu'))

# Capa de salida
cnn_model2.add(Dense(18, activation='softmax'))  # 18 clases de salida

# Compilar el segundo modelo
cnn_model2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# ----------CONFIGURAR TENSORBOARD-------------

# Definir los directorios de los logs para TensorBoard
log_dir_cnn_model = "logs/cnn_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir_cnn_model2 = "logs/cnn_model2/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Crear callbacks para TensorBoard
tensorboard_callback_cnn = TensorBoard(log_dir=log_dir_cnn_model, histogram_freq=1)
tensorboard_callback_cnn2 = TensorBoard(log_dir=log_dir_cnn_model2, histogram_freq=1)

# sasa

In [ ]:
# ----------ENTRENAMIENTO DEL PRIMER MODELO-------------

# Entrenar el primer modelo con TensorBoard
history_cnn = cnn_model.fit(
    X,
    y_train_one_hot,
    epochs=25,
    batch_size=32,
    validation_data=(X_t, y_test_one_hot),
    callbacks=[tensorboard_callback_cnn]
)

# Evaluar el primer modelo
cnn_model.evaluate(X_t, y_test_one_hot)

In [ ]:
# ----------ENTRENAMIENTO DEL SEGUNDO MODELO-------------

# Entrenar el segundo modelo con TensorBoard
history_cnn2 = cnn_model2.fit(
    X,
    y_train_one_hot,
    epochs=25,
    batch_size=32,
    validation_data=(X_t, y_test_one_hot),
    callbacks=[tensorboard_callback_cnn2]
)

# Evaluar el segundo modelo
cnn_model2.evaluate(X_t, y_test_one_hot)

In [ ]:
# ----------VISUALIZAR TENSORBOARD-------------

# Cargar la extensión de TensorBoard en Google Colab
%load_ext tensorboard

# Ejecutar TensorBoard
%tensorboard --logdir logs

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues, figsize=(10, 8)):
    """
    Esta función imprime y dibuja la matriz de confusión.
    La normalización se puede aplicar estableciendo `normalize=True`.
    `figsize` controla el tamaño de la imagen.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    # Configurar el tamaño del gráfico
    plt.figure(figsize=figsize)

    # Mostrar la matriz de confusión
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# Calcular la matriz de confusión
y_pred = cnn_model.predict(X_t)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_one_hot, axis=1)

confusion_mtx = confusion_matrix(y_true_classes, y_pred_classes)

# Obtener las etiquetas de las clases
class_labels = list(MAP_CHARACTERS.values())

# Mostrar la matriz de confusión con etiquetas y un tamaño de gráfico más grande
plot_confusion_matrix(confusion_mtx, classes=class_labels, title='Confusion matrix', figsize=(12, 10))


# Guardar el Modelo


In [ ]:

# Guardar el primer modelo
cnn_model.save('cnn_model.h5')

# Guardar el segundo modelo
cnn_model2.save('cnn_model2.h5')


# Probar el Modelo

In [ ]:
test_image_folder = '/content/drive/MyDrive/Colab Notebooks/simpsons_imagenes/Simpson'
!ls /content/drive/MyDrive/Colab\ Notebooks/simpsons_imagenes/Simpson

In [ ]:
# prompt: Carga el modelo, procesa las imágenes para testear (de una carpeta con esta ruta drive/MyDrive/Colab Notebooks/simpsons_ds/simpsons_test), predice la clase de cada imagen y muestras los resultados

import cv2
import numpy as np
from tensorflow.keras.models import load_model
from google.colab import drive

# Montar Google Drive
# drive.mount('/content/drive')

# Cargar el modelo entrenado
model = load_model('cnn_model.h5')  # O 'cnn_model2.h5' si quieres usar el segundo modelo

# Ruta de la carpeta con las imágenes de prueba
test_image_folder =  '/content/drive/MyDrive/Colab Notebooks/simpsons_imagenes/Simpson'

# Procesar las imágenes de prueba
def process_test_images(folder_path):
    images = []
    image_paths = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
            img = img.reshape(IMG_SIZE, IMG_SIZE, 1)    # Cambiar forma para CNN
            images.append(img)
            image_paths.append(img_path)  # Guardar las rutas de las imágenes
    return np.array(images), image_paths

# Cargar y procesar las imágenes de prueba
X_test_custom, image_paths = process_test_images(test_image_folder)
X_test_custom = X_test_custom / 255.0  # Normalizar las imágenes

# Realizar la predicción
predictions = model.predict(X_test_custom)

# Mostrar los resultados con la ruta y la clase predicha
for i, prediction in enumerate(predictions):
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha
    predicted_character = MAP_CHARACTERS[predicted_class]  # Mapear a personaje
    print(f"Imagen: {image_paths[i]}")
    print(f"Personaje predicho: {predicted_character}")
    print("-------------")


In [ ]:
# prompt: eneseñame las imagenes predichas

import numpy as np
import matplotlib.pyplot as plt

# Mostrar las imágenes predichas
for i, prediction in enumerate(predictions):
    predicted_class = np.argmax(prediction)
    predicted_character = MAP_CHARACTERS[predicted_class]
    plt.imshow(X_test_custom[i].reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
    plt.title(f"Predicted: {predicted_character}")
    plt.show()


In [ ]:
# prompt: Carga el modelo, procesa las imágenes para testear (de una carpeta con esta ruta drive/MyDrive/Colab Notebooks/simpsons_ds/simpsons_test), predice la clase de cada imagen y muestras los resultados

import cv2
import numpy as np
from tensorflow.keras.models import load_model
from google.colab import drive

# Montar Google Drive
# drive.mount('/content/drive')

# Cargar el modelo entrenado
model = load_model('cnn_model2.h5')  # O 'cnn_model2.h5' si quieres usar el segundo modelo

# Ruta de la carpeta con las imágenes de prueba
test_image_folder =  '/content/drive/MyDrive/Colab Notebooks/simpsons_imagenes/Simpson'

# Procesar las imágenes de prueba
def process_test_images(folder_path):
    images = []
    image_paths = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
            img = img.reshape(IMG_SIZE, IMG_SIZE, 1)    # Cambiar forma para CNN
            images.append(img)
            image_paths.append(img_path)  # Guardar las rutas de las imágenes
    return np.array(images), image_paths

# Cargar y procesar las imágenes de prueba
X_test_custom, image_paths = process_test_images(test_image_folder)
X_test_custom = X_test_custom / 255.0  # Normalizar las imágenes

# Realizar la predicción
predictions = model.predict(X_test_custom)

# Mostrar los resultados con la ruta y la clase predicha
for i, prediction in enumerate(predictions):
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha
    predicted_character = MAP_CHARACTERS[predicted_class]  # Mapear a personaje
    print(f"Imagen: {image_paths[i]}")
    print(f"Personaje predicho: {predicted_character}")
    print("-------------")


In [ ]:
# prompt: eneseñame las imagenes predichas

import numpy as np
import matplotlib.pyplot as plt

# Mostrar las imágenes predichas
for i, prediction in enumerate(predictions):
    predicted_class = np.argmax(prediction)
    predicted_character = MAP_CHARACTERS[predicted_class]
    plt.imshow(X_test_custom[i].reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
    plt.title(f"Predicted: {predicted_character}")
    plt.show()


# Exportación

In [ ]:
!pip install tensorflowjs

In [ ]:
!mkdir carpeta_salida

In [ ]:
!tensorflowjs_converter --input_format keras cnn_model2.h5 carpeta_salida

In [ ]:
from google.colab import drive
drive.mount('/content/drive')